# Extracts layer names from an input DWG file and stores them in a text file.


This example executes an AutoCAD plug-in that contains a command named **"LISTLAYERS"**, which extracts layer names from an input drawing file and saves them to a text file. It follows the official Autodesk tutorial: [Task 3 – Upload AppBundle](https://aps.autodesk.com/en/docs/design-automation/v4/tutorials/autocad/task3-upload-appbundle/).

In the `examples\AutoCAD_01_list_layers\files` folder, you can find the sample `.DWG` file and the bundle `ListLayers.zip`. You can also refer to the official Autodesk GitHub repository: [aps-listlayers-designautomation](https://github.com/autodesk-platform-services/aps-listlayers-designautomation).
n

<p align="center">
  <img src="./files/cover.png" alt="Cover Image">
</p>

## Step 1: Setup and Import Libraries

Import all necessary libraries and modules.

In [ ]:
import os
import json
import logging
import uuid
from pathlib import Path
from dotenv import load_dotenv

from aps_automation_sdk.classes import (
    Activity,
    ActivityInputParameter,
    ActivityOutputParameter,
    AppBundle,
    WorkItem
)

from aps_automation_sdk.utils import (
    delete_activity,
    delete_appbundle,
    get_token,
    set_nickname
)

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

## Step 2: Load Environment Variables and Configure Authentication

Load credentials from environment variables and get an authentication token.

**Prerequisites:**
1. Create an APS application to get your credentials. Follow these tutorials:
   - Create App: https://aps.autodesk.com/en/docs/oauth/v2/tutorials/create-app
   - Design Automation Setup: https://aps.autodesk.com/en/docs/design-automation/v3/developers_guide/aliases-and-ids/ 

2. Create a `.env` file in the project root with your credentials (see `.env.sample` for reference)

**Note on Nicknames:** The nickname must be unique across all APS apps globally. If you encounter errors, try using alphanumeric unique values. Choose a distinctive nickname to avoid conflicts.

In [ ]:
load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID", "")
CLIENT_SECRET = os.getenv("CLIENT_SECRET", "")

# Get authentication token
token = get_token(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)

# Set nickname. If the app already has a nickname, the previous one will be returned.
nickname = set_nickname(token, "myUniqueNickNameHere")

print(f"Authentication successful. Nickname: {nickname}")

## Step 3: Define Project Constants

Set up all the constants needed for the AutoCAD list layers workflow.

**Note:** The input DWG file and compiled app bundle can be found in `examples\AutoCAD_01_list_layers\files`

In [ ]:
# Define constants
app_bundle_name = "ListLayers"
activity_name = "ListLayersActivity"
alias = "prod"
bucket_key = uuid.uuid4().hex
zip_path = Path.cwd() / "files" / "ListLayers.zip"

# Create full aliases
appbundle_full_alias = f"{nickname}.{app_bundle_name}+{alias}"
activity_full_alias = f"{nickname}.{activity_name}+{alias}"

print(f"App Bundle: {appbundle_full_alias}")
print(f"Activity: {activity_full_alias}")

## Step 4: Create and Deploy App Bundle

Register and upload the app bundle that contains the list layers logic.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/autocad/task3-upload-appbundle/

In [ ]:
bundle = AppBundle(
    appBundleId=app_bundle_name,
    engine="Autodesk.AutoCAD+24_3",
    alias=alias,
    zip_path=str(zip_path),
    description="ListLayers Bundle",
)

bundle.deploy(token)
print("App bundle deployed successfully!")

## Step 5: Define Activity Parameters

Create input and output parameters for the activity.

**Reference:** https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/autocad/task3-upload-appbundle/ 

In [ ]:
# Input: DWG file
input_dwg = ActivityInputParameter(
    name="InputDwg",
    localName="Input.dwg",
    verb="get",
    description="Input drawing File",
    required=True,
    is_engine_input=True,
    bucketKey=bucket_key,
    objectKey="Input.dwg",
)

# Output: Text file with layer names
output_file = ActivityOutputParameter(
    name="result",
    localName="layers.txt",
    verb="put",
    description="Results",
    bucketKey=bucket_key,
    objectKey="layers.txt",
)

## Step 6: Create and Deploy Activity

Create the activity that links the app bundle with the parameters.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/autocad/task4-publish-activity/ 

In [ ]:
activity = Activity(
    id=activity_name,
    parameters=[input_dwg, output_file],
    engine="Autodesk.AutoCAD+24_3",
    appbundle_full_name=appbundle_full_alias,
    description="Extracts layer names from an input drawing file and saves them to a text file",
    alias=alias,
    script='(command \"LISTLAYERS\")\n'
)

# Set the AutoCAD command
activity.set_autocad_command_line()
activity.deploy(token=token)
print("Activity deployed successfully!")

# Extracts layer names from an input DWG and stores them in a text file. 

This notebook demonstrates how to run an AutoCAD plug-in. This plug-in contains a command called "LISLAYERS".

**Reference:** https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/autocad/about_this_tutorial/


## Step 7: Upload Input Files

Upload the DWG file to Object Storage Service (OSS).

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/autocad/task5-prepare_cloud_storage/

In [ ]:
input_dwg_path = Path.cwd() / "files" / "Mechanical - Multileaders.dwg"
input_dwg.upload_file_to_oss(file_path=str(input_dwg_path), token=token)
print(f"Input DWG file uploaded: {input_dwg_path.name}")

## Step 8: Create and Execute Work Item

Create a work item with all parameters and execute it.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/autocad/task6-post-workitem/

In [ ]:
work_item = WorkItem(
    parameters=[input_dwg, output_file],
    activity_full_alias=activity_full_alias
)
print("Work item created. Starting execution...")

## Step 10: Poll Work Item Status

Monitor the work item execution until completion (max 10 minutes).

In [ ]:
status_resp = work_item.execute(token=token, max_wait=600, interval=10)
last_status = status_resp.get("status", "")

print(f"Work item completed with status: {last_status}")
print(f"Full response: {json.dumps(status_resp, indent=2)}")

## Step 11: Download Results

If successful, download the results.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/autocad/task7-download-results/

In [ ]:
if last_status == "success":
    out_dir = Path.cwd() / "files" / "output"
    out_dir.mkdir(parents=True, exist_ok=True)
    out_txt = out_dir / "result.txt"
    output_file.download_to(output_path=str(out_txt), token=token)
    print(f"✅ Download successful: {out_txt}")
else:
    print(f"❌ Work item failed or did not complete successfully. Status: {last_status}")

## Step 12: Cleanup Resources

Delete the activity and app bundle to clean up resources.

In [ ]:
print("Cleaning up: deleting activity and app bundle...")

try:
    delete_activity(activityId=activity_name, token=token)
    print(f"✅ Activity deleted: {activity_full_alias}")
except Exception as e:
    print(f"⚠️ Failed to delete activity: {e}")

try:
    delete_appbundle(appbundleId=app_bundle_name, token=token)
    print(f"✅ App bundle deleted: {app_bundle_name}")
except Exception as e:
    print(f"⚠️ Failed to delete app bundle: {e}")

print("\n🎉 AutoCAD list layers workflow completed!")